In [2]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import scipy.io as io
from scipy.io import loadmat
from scipy.io import savemat
import torch.utils.data as data_utils
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import os
from scipy.stats import entropy
import time
from random import random
from random import randint
import argparse

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [3]:
from torch.utils.data import Dataset,TensorDataset, DataLoader
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix
# Load feature data
def load_data(path,file):
    name=path+file
    m=loadmat(name)
    x=torch.tensor(m['feature']).to(device)
    return x.float()

def load_data_all(path,names):
    emptyx=1
    nc=len(names)
    for j in range(nc):
        x=load_data(path,names[j])
        x=x.to(device)
        if emptyx:
            allx=x
            emptyx=0
        else:
            allx=torch.cat((allx,x),dim=0)
        l=j
        if j==0:
            y=torch.zeros(x.shape[0]).to(device)+l
        else:
            y=torch.cat((y,torch.zeros(x.shape[0]).to(device)+l))
        if (j%200==199)|(j==nc-1):
            print(j+1)   
    return allx,y


def tr(a,b):
    dab=torch.diagonal(a@b, dim1=-1)
    return torch.sum(dab,dim=1)

def tr1(a,b):
    return torch.sum(a.t()*b,dim=[1,2])

def computeClassPPCAs(path,names,device,q=200):
    nc = len(names)
    la=0.01
    for i in range(nc):
        name = '%sfeatures_640/%s.mat'%(path, names[i])
        m=loadmat(name)
        x=torch.tensor(m['feature']).float().to(device)
        if i==0:
            print(x.shape)
        mx=torch.mean(x,dim=0)
        xx=(x-mx).t()@(x-mx)/x.shape[0]
        u,s,v = torch.linalg.svd(xx)
        s[s < la] = la
        Sigmai=(u*s)@u.t()
        name=path+'Model/'+names[i]+'.pth'
        torch.save({'mx':mx,'L':u[:,0:q].t(),'S':s[0:q],'Sigma':Sigmai},name)    
        if (i%200==199)|(i==nc-1):
            print(i+1)   
        #self.sxx[i] = para_dict['sxx']
    print('done computing class models')
    #print(mu.shape,L.shape,S.shape)

def loadClassPPCAs(path,names,device,q, loadSigma=0):
    nc = len(names)
    la=0.01
    for i in range(nc):
        full_feature_dir = '%s/%s.pth'%(path, names[i])
        para_dict = torch.load(full_feature_dir)
        mx=para_dict['mx'].squeeze()
        Li=para_dict['L']
        Si=para_dict['S'].squeeze()
        if i==0:
            print(para_dict.keys())
            print(Li.shape,Si.shape)
            d=mx.shape[0]
            mu=torch.zeros(nc,d,device=device)
            L=torch.zeros(nc,q,Li.shape[1],device=device)
            S=torch.zeros(nc,Si.shape[0],device=device)
            if loadSigma:
                Sigma=torch.zeros(nc,d,d,device=device)
        mu[i,:]=mx.to(device)
        L[i,:,:]=Li[:q,:].to(device)
        if loadSigma:
            if 'Sigma'in para_dict.keys():
                Sigma[i,:,:] = para_dict['Sigma'].to(device)
            else:
                Si=Si**2
                Sigma[i,:,:] = (Li.t()*Si)@Li+la*torch.eye(d,device=device)
        S[i,:]=Si.to(device)
        #self.sxx[i] = para_dict['sxx']
    print('done loading classes')
    #print(mu.shape,L.shape,S.shape)
    if not loadSigma:
        return mu,L,S
    return mu,L,S,Sigma

def response(x1, mu, L, S,la):
    # response using L,S (fast), using the math computation above
    # xSigma^-1 x = x^Tx/lambda - u^Tu/lambda where u = S/(S^2+lambda)^0.5Lx
    x = x1 - mu
    if S.shape[0]==0:
        return torch.sum(x ** 2, dim=1)
    Lx = x @ L.t()
    Qx = ( torch.sqrt(S) / torch.sqrt(S  + la)).unsqueeze(0) * Lx
    return (torch.sum(x ** 2, dim=1) - torch.sum(Qx ** 2, dim=1)) / la#+sum(torch.log(S+la))

def classify(x,mx,L,S,q,la):
    nj=x.shape[0]
    nc=mx.shape[0]
    out=torch.zeros(nj,nc,device=device)
    for i in range(nc):
        r=response(x,mx[i,:],L[i,:q,:],S[i,:q],la)
        out[:,i] = r
    py=torch.argmin(out,dim=1)
    return py

def B_distance(mx1,Sigma1,mx2,Sigma2):
    mu=mx1-mx2
    Sigma=torch.linalg.inv((Sigma2+Sigma1)/2)
    #print(Sigma.shape,mu.shape)
    Simu=Sigma@mu.unsqueeze(2)
    #print(Simu.shape)
    dis= mu.unsqueeze(1)@Simu
    return dis.squeeze()/8

def test_PPCA(test_loader,mu,L,S,q=20,la=0.01):
    nc=mu.shape[0]
    total = 0
    wrong = 0
    total_wrong = 0
    start = time.time()
    for data in test_loader:
        features, labels = data
        labels = torch.squeeze(labels)
        features, labels = features.to(device), labels.to(device)
        label = classify(features, mu, L, S, q, la)

        total_wrong += torch.sum(label != labels)
        # wrong_list1.append(wrong/len(py))

        total += len(labels)
    end = time.time()
        #print('Computation time is', end - start)
    #print('super acc:',100*super_correct/total)
    acc=1-total_wrong.item()/total
    print('q:%d,accuracy:%.4f,t:%.1f' %(q,acc,end - start))  
    return acc

def generate_pnames(names,nruns=10,nn=100):
    pnames={}
    for it in range(nruns):    
        names=np.random.permutation(names)
        pnames[it+1]=names[:nn]
    torch.save(pnames,'pnames.pth')

def load_subsamples(path,names,nsamples):
    emptyx=1
    nc=len(names)
    
    for i in range(nc):
        xi=load_data(path,names[i])
        xi=xi.to(device)
        if emptyx:
            d=xi.shape[1]
            x=torch.zeros((nc*nsamples,d),device=device)
            print(x.shape)
            y=torch.zeros(nc*nsamples,device=device)
            emptyx=0
        j=torch.randperm(xi.shape[1],device=device)
        x[(i*nsamples):((i+1)*nsamples),:]=xi[j[:nsamples],:]
        y[(i*nsamples):((i+1)*nsamples)]=torch.zeros(nsamples,device=device)+i
        if (i%200==199)|(i==nc-1):
            print(i+1)   
    return x,y

def subsample_data(path,names):
    traind={}
    for it in range(5):
        x,y=load_subsamples(path+'features_640/',names,20)
        traind[it+1]=(x,y)
    torch.save(traind,'c:/training/imagenet/train_raw.pth')
    
def generateSuper(mu,sigma):
    # the superclass mean and sigma
    mu,sigma=mu.to(device),sigma.to(device)
    superMu=torch.mean(mu,dim=0)
    s_mu=mu-superMu
    n=mu.shape[0]
    d=mu.shape[1]
    xx=s_mu.unsqueeze(2)@s_mu.unsqueeze(1)
    superSigma=torch.mean(xx,dim=0)+torch.mean(sigma,dim=0)
    return superMu,superSigma

def updateParameters(ind, smu, sSigma, mu_p, Sigma_p):
    nsc=smu.shape[0]
    for i in range(nsc):
        mui,si=generateSuper(mu_p[ind == i,:],Sigma_p[ind == i,:,:])
        smu[i,:] = mui
        sSigma[i,:,:]=si
    return smu, sSigma

def updateLS(Sigma,q=20, la=0.01):
    u, s, v = torch.linalg.svd(Sigma, full_matrices=True)
    L=v[:,:q, :]
    S = s[:,:q]
    return L,S

def getIndex(ind,nsc,nc):
    indexDict={}
    for i in range(nsc):
        indexDict[i] = []
    for i in range(nc):
        indexDict[int(ind[i])].append(i)
    return indexDict


#path='d:/datasets/Imagenet/'
path = 'd:/datasets/ILSVRC2016/'
name = path+'train.txt'
with open(name, 'r') as f:
    names = f.read().splitlines()
#names=pnames[it+1]
nc=len(names)
print(nc)
#computeClassPPCAs(path,names,device)
#mu,L,S,Sigma=loadClassPPCAs(path+'model_640',names,device)
#xt,yt=load_data_all(path+'features_val_640/',names)
#print(xt.shape,yt.shape)
#data=TensorDataset(xt,yt.long())
#test_loader=DataLoader(data,batch_size=10000,shuffle=False)

1000


In [4]:
# load image class mdoels

def loadClassPPCAs(path,names,device,q,loadSigma=0):
    nc = len(names)
    la=0.01
    for i in range(nc):
        full_feature_dir = '%s/%s.pth'%(path, names[i])
        para_dict = torch.load(full_feature_dir)
        mx=para_dict['mx'].squeeze()
        Li=para_dict['L']
        Si=para_dict['S'].squeeze()
        if i==0:
            print(para_dict.keys())
            print(Li.shape,Si.shape)
            d=mx.shape[0]
            mu=torch.zeros(nc,d,device=device)
            L=torch.zeros(nc,Li.shape[0],Li.shape[1],device=device)
            S=torch.zeros(nc,Si.shape[0],device=device)
            if loadSigma:
                Sigma=torch.zeros(nc,d,d,device=device)
        mu[i,:]=mx
        L[i,:,:]=Li
        if loadSigma:
            if 'Sigma'in para_dict.keys():
                Sigma[i,:,:] = para_dict['Sigma']
            else:
                Si=Si**2
                Sigma[i,:,:] = (Li.t()*Si)@Li+la*torch.eye(d,device=device)
        S[i,:]=Si
        #self.sxx[i] = para_dict['sxx']
        if (i%1000==999)|(i==nc-1):
            print(i+1)   
    print('done loading classes')
    #print(mu.shape,L.shape,S.shape)
    if not loadSigma:
        return mu,L,S
    return mu,L,S,Sigma

mu,L,S=loadClassPPCAs(path+'model_640',names,device,q=50)
#mu,L,S=loadClassPPCAs('e:/datasets/imagenet/model_640',names,device,q=50)
#mu,L,S,Sigma=loadClassPPCAs(path+'model_640',names,device,q=50,loadSigma=1)


dict_keys(['mx', 'L', 'S', 'Sigma'])
torch.Size([200, 640]) torch.Size([200])
1000
done loading classes


In [9]:
torch.save({'mu':mu,'L':L,'S':S},'c:/training/Imagenet/model_1k_q50.pth')

In [14]:
m=torch.load('c:/training/Imagenet/model_10k_q50.pth')
mu=m['mu']
L=m['L']
S=m['S']

In [7]:
# load test data
m=torch.load(path+'features_val_640.pth')
xt=m['xt']
yt=m['yt']
data=TensorDataset(xt,yt.long())
test_loader=DataLoader(data,batch_size=50000,shuffle=False)

In [48]:
torch.save({'xt':xt,'yt':yt},'d:/datasets/Imagenet/test_1k.pth')

In [5]:
xt,yt=load_data_all(path+'features_val_640/',names)

200
400
600
800
1000


In [8]:
# Evaluate HPPCA and HPPCA raw

import pandas as pd

def classifyT(x, mx, L, S,q,la,T):
    nj = x.shape[0]

    nc = mx.shape[0]
    out = torch.zeros(nj, nc, device=device)
    for i in range(nc):
        r = response(x, mx[i, :], L[i, :q, :], S[i, :q],la)
        out[:, i] = r
        sorte, indices = torch.sort(out,dim=1)
    return indices[:,:T]

def classify_ori(x, mx, L, S, supermask, q,la=0.01):
    nj = x.shape[0]

    nc = mx.shape[0]
    out = 100000 * torch.ones(nj, nc, device=device)
    for i in range(nc):
        r = response(x[torch.gt(supermask[:, i], 0)], mx[i, :].to(device), L[i, :q, :].to(device), S[i, :q].to(device), la)
        out[torch.gt(supermask[:, i], 0), i] = r

    py = torch.argmin(out, dim=1)
    return py

def test_HPPCA(model,mu,L,S,test_loader,T=4,q=100,r=100):
    nc=model['nc']
    nsc=model['nsc']
    ind=model['ind']
    indexDict=model['indexDict']
    total = 0
    wrong = 0
    total_wrong = 0
    super_correct=0
    start = time.time()
    density = 0
    for data in test_loader:
        features, labels = data
        labels = torch.squeeze(labels)
        features, labels = features.to(device), labels.to(device)
        py = classifyT(features, model['smu'], model['sL'], model['sS'],r, la=0.01, T=T)
        superlabels=ind[labels]
        #print(model['smu'].shape,py.shape)
        super_correct += torch.sum(torch.max((py-superlabels.view(-1,1)==0).float(),dim=1)[0])
        supermask = torch.zeros((len(py), nc), device=device)
        i = 0
        for i in range(len(py)):
            for j in range(T):
                superlabel = py[i][j]
                superIndex = indexDict[int(superlabel)]

                supermask[i][superIndex] = 1

        density += supermask.sum()
        label = classify_ori(features, mu, L, S, supermask,q, la=0.01)

        total_wrong += torch.sum(label != labels)
        # wrong_list1.append(wrong/len(py))

        total += len(py)
        end = time.time()
        #print('Computation time is', end - start)
    #print('super acc:',100*super_correct/total)
    density+=nsc*total
    density=density.item() / (total * nc)
    speedup=1/density
    acc=1-total_wrong.item()/total
    super_acc=super_correct.item()/total
    print('T:%d,q:%d,r:%d,nsc:%d,density:%.3f,speedup:%.1f,accuracy:%.3f,super_acc:%.3f' %(T,q,r,nsc,density,speedup,acc,super_acc))
    
    result={'density':density,'speedup':speedup,'acc':acc,'super_acc':super_acc}
    return result

pnames=torch.load('pnames100.pth')
start=1
torch.cuda.empty_cache()
for i in range(0,5):
    if 0: # For ImageNet-100
        names=pnames[i+1]
        mu,L,S=loadClassPPCAs('d:/datasets/ILSVRC2016/model_640',names,device,q=200)
        xt,yt=load_data_all('d:/datasets/ILSVRC2016/features_val_640/',names)
        print(i,nc,xt.shape,yt.shape)
        data=TensorDataset(xt,yt.long())
        test_loader=DataLoader(data,batch_size=10000,shuffle=False)
    nc=len(names)
    for nsc in [33]:
        name='c:/training/ppca/hppca_640_1k_%d_%d.pth'%(nsc,i+1)
        #name='raw_640_10k_%d_%d.pth'%(nsc,i+1)
        model=torch.load(name)
        model['sL']=model['sL'].to(device)
        model['sS']=model['sS'].to(device)
        model['smu']=model['smu'].to(device)
        torch.cuda.empty_cache()
        for r in [50]:#[0,10,20,50,100,200]:
            q=r
            for T in [1,5]:
                result=test_HPPCA(model,mu,L,S,test_loader,T=T,q=q,r=r)
                if start:
                    start=0
                    res={'T':[],'q':[],'r':[],'i':[],'nsc':[]}
                    for k in result.keys():
                        res[k]=[]
                res['nsc'].append(model['nsc'])
                res['T'].append(T)
                res['q'].append(q)
                res['r'].append(r)
                res['i'].append(i+1)
                for k in result.keys():
                    res[k].append(result[k])
                #print(res)
        df=pd.DataFrame(res)
        print(df)
        name='res_hppca.csv'
        df.to_csv(name)

T:1,q:50,r:50,nsc:33,density:0.081,speedup:12.4,accuracy:0.673,super_acc:0.831
T:5,q:50,r:50,nsc:33,density:0.214,speedup:4.7,accuracy:0.734,super_acc:0.979
   T   q   r  i  nsc   density    speedup      acc  super_acc
0  1  50  50  1   33  0.080550  12.414723  0.67272    0.83092
1  5  50  50  1   33  0.213927   4.674489  0.73390    0.97928
T:1,q:50,r:50,nsc:33,density:0.073,speedup:13.8,accuracy:0.668,super_acc:0.819
T:5,q:50,r:50,nsc:33,density:0.215,speedup:4.6,accuracy:0.733,super_acc:0.981
   T   q   r  i  nsc   density    speedup      acc  super_acc
0  1  50  50  1   33  0.080550  12.414723  0.67272    0.83092
1  5  50  50  1   33  0.213927   4.674489  0.73390    0.97928
2  1  50  50  2   33  0.072659  13.762939  0.66784    0.81862
3  5  50  50  2   33  0.215158   4.647740  0.73346    0.98138
T:1,q:50,r:50,nsc:33,density:0.072,speedup:13.8,accuracy:0.673,super_acc:0.822
T:5,q:50,r:50,nsc:33,density:0.212,speedup:4.7,accuracy:0.734,super_acc:0.979
   T   q   r  i  nsc   density   

In [14]:
# not used
def computeDisMatrix(mu,Sigma):
    print('Computing distance matrix')
    nc=mu.shape[0]
    mat=torch.zeros((nc,nc),device=device)
    with torch.no_grad():
        for i in range(1, nc):
            di=B_distance(mu[i,:],Sigma[i,:,:],mu[:i,:],Sigma[:i,:])
            mat[i, :i] = di

    for i in range(0, nc - 1):
        mat[i, i+1:] = mat[i+1:, i]
    return mat

dist=computeDisMatrix(mu,Sigma)
print(dist.shape,torch.max(dist))
torch.save(dist,'d:/Datasets/ILSVRC2016/dist.pth')

Computing distance matrix
torch.Size([1000, 1000])


In [7]:
dist=torch.load('d:/Datasets/ILSVRC2016/Model_640/dist.pth')

In [10]:
# Train HPPCA

def KL(mu_q,invSigma_q,mu_p,Sigma_p):
    with torch.no_grad():
        n=mu_p.shape[0]
        d=invSigma_q.shape[1]
        mu=mu_p-mu_q
        muSimu=(mu.unsqueeze(1)@invSigma_q@mu.unsqueeze(2)).squeeze()
        #score2=torch.sum(torch.diagonal(invSigma_q@Sigma_p, dim1=-1),dim=1)
        tr=torch.sum(invSigma_q.t()*Sigma_p,dim=[1,2])
        logdets=torch.logdet(Sigma_p)+torch.logdet(invSigma_q)
    return (muSimu+tr-logdets-d)/2

def kMeansInit(disMatrix, nsc):
    # k-Means++ initialization
    # uses a precomputed distance matrix (slow)
    with torch.no_grad():
        nc=disMatrix.shape[0]
        disM = torch.where(disMatrix > 0, disMatrix, torch.zeros(disMatrix.shape, device=device))
        init_list = torch.zeros(nsc, device=device).long()

        init_list[0] = randint(0, nc - 1)
        row = disM[init_list[0],:]
        for i in range(1, nsc):
            ind = np.random.choice(nc, p=(row / sum(row)).cpu().numpy())
            init_list[i] = ind
            row, _ = torch.min(torch.stack((row, disM[ind,:]), dim=0), dim=0)
    return torch.sort(init_list)[0]

def B_distance(mx1,Sigma1,mx2,Sigma2):
    # Bhattacharyya distances
    mu=mx1-mx2
    Sigma=torch.linalg.inv((Sigma2+Sigma1)/2)
    #print(Sigma.shape,mu.shape)
    Simu=Sigma@mu.unsqueeze(2)
    #print(Simu.shape)
    dis= mu.unsqueeze(1)@Simu
    return dis.squeeze()/8


def B_distances(mx1,Sigma1,loader):
    # Bhattacharyya distances using a loader
    dis=[]
    mx1=mx1.to(device)
    Sigma1=Sigma1.to(device)
    for mui,sigmai in loader:
        mui,sigmai=mui.to(device),sigmai.to(device)
        mu=mx1-mui
        Sigma=sigmai+Sigma1
        Sigma=torch.linalg.inv(Sigma/2)
        Simu=Sigma@mu.unsqueeze(2)
        #print(Simu.shape)
        disi= mu.unsqueeze(1)@Simu
        dis.append(disi.squeeze())
    dis=torch.cat(dis)
    dis[dis<0]=0
    return dis/8

def kMeansInit1(loader, nsc):
    # k-Means++ initialization
    # computes only what distances it needs
    print('init')
    t0=time.time()
    nc=mu.shape[0]
    init_list = torch.zeros(nsc, device=device).long()
    ind=randint(0, nc - 1)
    init_list[0] = ind
    row = B_distances(mu[ind,:],Sigma[ind,:,:],loader)
    for i in range(1, nsc):
        ind = np.random.choice(nc, p=(row / sum(row)).cpu().numpy())
        init_list[i] = ind
        di=B_distances(mu[ind,:],Sigma[ind,:,:],loader)
        row, _ = torch.min(torch.stack((row, di), dim=0), dim=0)
        if i%5==4:
            print(i)
    print('done in %.1fs'%(time.time()-t0))
    return torch.sort(init_list)[0]

def generateSuper(mu,sigma):
    # the superclass mean and sigma from Theorem 2
    mu,sigma=mu.to(device),sigma.to(device)
    superMu=torch.mean(mu,dim=0)
    s_mu=mu-superMu
    n=mu.shape[0]
    d=mu.shape[1]
    xx=s_mu.unsqueeze(2)@s_mu.unsqueeze(1)
    superSigma=torch.mean(xx,dim=0)+torch.mean(sigma,dim=0)
    return superMu,superSigma

def computeKL(disMatrix,mu,invSigma, mu_p, Sigma_p):
    torch.cuda.empty_cache()
    nc=mu_p.shape[0]
    nsc=mu.shape[0]
    with torch.no_grad():
        for i in range(nsc):
             #   for j in range(int(self.nc/500)):        
            disMatrix[:, i] = KL(mu[i],invSigma[i],mu_p,Sigma_p)
        score, ind = torch.min(disMatrix, dim=1)
        ind = ind
        loss = torch.sum(score)
        print('loss is', loss)
    return ind

def max_cluster(ind):
    nsc=torch.max(ind)+1
    ne=torch.zeros(nsc).long()
    for i in range(nsc):
        ne[i]=torch.sum(ind==i)
    return torch.max(ne).item()

def updateParameters(ind, smu, sSigma, mu_p, Sigma_p):
    nsc=smu.shape[0]
    for i in range(nsc):
        mui,si=generateSuper(mu_p[ind == i,:],Sigma_p[ind == i,:,:])
        smu[i,:] = mui
        sSigma[i,:,:]=si
    return smu, sSigma

def updateLS(Sigma,q=20, la=0.01):
    u, s, v = torch.linalg.svd(Sigma, full_matrices=True)
    L=v[:,:q, :]
#                self.L[i,:,:] = Li
    S = s[:,:q]
#    Sigma = (L.permute(0,2,1)*S.unsqueeze(1))@L+la*torch.eye(Sigma.shape[1],device=v.device).unsqueeze(0)
#    invSigma = torch.linalg.inv(Sigma)
    return L,S#,Sigma,invSigma
    
def getIndex(ind,nsc,nc):
    indexDict={}
    for i in range(nsc):
        indexDict[i] = []
    for i in range(nc):
        indexDict[int(ind[i])].append(i)
    return indexDict


# cluster the class Gaussians
pnames=torch.load('pnames100.pth')
n_iter=4
for it in range(5):
    if 1: # For ImageNet-100
        names=pnames[it+1]
        nc=len(names)
        mu,L,S,Sigma=loadClassPPCAs('d:/datasets/ILSVRC2016/model_640',names,device,q=50,loadSigma=1)
    data=TensorDataset(mu,Sigma)
    loader=DataLoader(data,batch_size=200,shuffle=False)
    d=Sigma.shape[1]
    for nsc in [10]:
        print(nsc,nc,d)
        t0=time.time()
        #init_list=selectInit(dist,nsc)
        init_list=kMeansInit1(loader,nsc)
        smu=mu[init_list,:].clone()
        sSigma=Sigma[init_list,:,:].clone()
        sinvSigma = torch.linalg.inv(sSigma)
        disMatrix = torch.zeros((nc, nsc), device=device, dtype=torch.float)
        for i in range(n_iter):
            ind=computeKL(disMatrix,smu,sinvSigma,mu,Sigma)
            print('max cluster is', max_cluster(ind))
            smu,sSigma=updateParameters(ind,smu,sSigma,mu,Sigma)
            sinvSigma = torch.linalg.inv(sSigma)
        sL,sS=updateLS(sSigma,200)
        indexDict=getIndex(ind,nsc,nc)
        model={'nc':nc,'nsc':nsc,'smu':smu,'sL':sL,'sS':sS,'ind':ind,'indexDict':indexDict}
        name='hppca_640_100_%d_%d.pth'%(nsc,it+1)
        torch.save(model,name)
        print('Finished training', time.time()-t0)

dict_keys(['mx', 'L', 'S', 'Sigma'])
torch.Size([200, 640]) torch.Size([200])
100
done loading classes
10 100 640
init
4
9
done in 0.6s
loss is tensor(8592.9102, device='cuda:0')
max cluster is 38
loss is tensor(2431.9058, device='cuda:0')
max cluster is 38
loss is tensor(2431.9058, device='cuda:0')
max cluster is 38
loss is tensor(2431.9058, device='cuda:0')
max cluster is 38
Finished training 2.0731048583984375
dict_keys(['mx', 'L', 'S', 'Sigma'])
torch.Size([200, 640]) torch.Size([200])
100
done loading classes
10 100 640
init
4
9
done in 0.5s
loss is tensor(7767.0073, device='cuda:0')
max cluster is 38
loss is tensor(2306.5996, device='cuda:0')
max cluster is 38
loss is tensor(2306.5996, device='cuda:0')
max cluster is 38
loss is tensor(2306.5996, device='cuda:0')
max cluster is 38
Finished training 1.9218406677246094
dict_keys(['mx', 'L', 'S', 'Sigma'])
torch.Size([200, 640]) torch.Size([200])
100
done loading classes
10 100 640
init
4
9
done in 0.5s
loss is tensor(9132.0146, devi

In [92]:

def KLs(mu_q,invSigma_q,loader):
    mu_q=mu_q.to(device)
    invSigma_q=invSigma_q.to(device)
    kls=[]
    d=invSigma_q.shape[1]
    ld=torch.logdet(invSigma_q)
    for mui,sigmai in loader:
        mui,sigmai=mui.to(device),sigmai.to(device)
        mu=mui-mu_q
        muSimui=(mu.unsqueeze(1)@invSigma_q@mu.unsqueeze(2)).squeeze()
        #score2=torch.sum(torch.diagonal(invSigma_q@Sigma_p, dim1=-1),dim=1)
        tri=torch.sum(invSigma_q.t()*sigmai,dim=[1,2])
        logdeti=torch.logdet(sigmai)+ld
        kls.append(muSimui+tri-logdeti)
    kls=torch.cat(kls)
    return (kls-d)/2

data=TensorDataset(mu,Sigma)
loader=DataLoader(data,batch_size=10,shuffle=False)
b1 = KLs(mu[0,:],Sigma[0,:,:],loader).cpu()
b2 = KL(mu[0,:],Sigma[0,:,:],mu,Sigma).cpu()
print(torch.max(torch.abs(b1-b2))/torch.max(torch.abs(b2)))

tensor(1.8836e-07)


In [11]:
# Train HPPCA raw
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import contingency_matrix
nsc=33
def HPPCA_raw(x,y,nsc,mu,Sigma,random_state):
    sc = KMeans(n_clusters=nsc, random_state=random_state, n_init="auto").fit(x.cpu().numpy())
    c=torch.tensor(contingency_matrix(y.cpu(), sc.labels_))
    ind=torch.argmax(c,dim=1).to(device)
    d=x.shape[1]
    smu=torch.zeros(nsc,d,device=device)
    sSigma=torch.zeros(nsc,d,d,device=device)
    smu,sSigma=updateParameters(ind, smu, sSigma, mu, Sigma)
    sL,sS=updateLS(sSigma,200)
    indexDict=getIndex(ind,nsc,nc)
    model={'nc':nc,'nsc':nsc,'smu':smu,'sL':sL,'sS':sS,'ind':ind,'indexDict':indexDict}
    return model
nsc=10
pnames=torch.load('pnames100.pth')
for it in range(5):
    if 1: # For ImageNet-100
        names=pnames[it+1]
        nc=len(names)
        mu,L,S,Sigma=loadClassPPCAs(path+'model_640',names,device,q=50,loadSigma=1)
    x,y=load_subsamples(path+'features_640/',names,20)
    t0=time.time()
    model=HPPCA_raw(x,y,nsc,mu,Sigma,it+1)
    name='raw_640_100_%d_%d.pth'%(nsc,it+1)
    torch.save(model,name)
    print('Finished training', time.time()-t0)

dict_keys(['mx', 'L', 'S', 'Sigma'])
torch.Size([200, 640]) torch.Size([200])
100
done loading classes
torch.Size([2000, 640])
100
Finished training 0.971869945526123
dict_keys(['mx', 'L', 'S', 'Sigma'])
torch.Size([200, 640]) torch.Size([200])
100
done loading classes
torch.Size([2000, 640])
100
Finished training 0.6351423263549805
dict_keys(['mx', 'L', 'S', 'Sigma'])
torch.Size([200, 640]) torch.Size([200])
100
done loading classes
torch.Size([2000, 640])
100
Finished training 0.6461451053619385
dict_keys(['mx', 'L', 'S', 'Sigma'])
torch.Size([200, 640]) torch.Size([200])
100
done loading classes
torch.Size([2000, 640])
100
Finished training 0.6441442966461182
dict_keys(['mx', 'L', 'S', 'Sigma'])
torch.Size([200, 640]) torch.Size([200])
100
done loading classes
torch.Size([2000, 640])
100
Finished training 0.6461443901062012
